In [1]:
from glob import glob
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [149]:

class CorrelationTables:
    def __init__(self, show_logs, show_graphs):
        print('Please add data primitives in format of data path')
        self.primitives_list = list()
        self.correlation_results = list()
        self.total_result = list()
        self.warning_logs = show_logs
        self.graphs = show_graphs

    def add_research_primitive(self, primitive_path, index_col=0, data_freq='1T'):
        primitive_name = primitive_path.split('/')[-1].split('.')[0]
        primitive_data = pd.read_csv(primitive_path, index_col=index_col)
        primitive_data.index = pd.to_datetime(primitive_data.index)
        if self.warning_logs:
            print(f'====    Added {primitive_name}  ====')
        self.primitives_list.append({"Primitive_name": primitive_name, "Primitive_content": primitive_data, "Primitive_volume": primitive_data.shape[0],
                                     "Primitive_Frequency": data_freq})
        del primitive_name
        del primitive_data

    def make_correlation_table(self, table_data_slices, show_logs=False):
        if self.warning_logs:
            print('=' * 10)
            print('Be careful. In one correlation table can be only dataframes with similar shapes')
        #TODO Need to make method to guarantee sync records from datetimes
        # Таким образом находятся пары между которыми можно будет считать корреляцию. Этот способ не очень хороший, но по крайней мере он практически полностью
        # гарантирует отсутствие сдвига во времени в кусках данных
        unique_shapes = np.unique(np.array([primitive["Primitive_volume"] for primitive in self.primitives_list]))
        for similar_shapes in unique_shapes:
            # Собираем список из пар имеющих одинаковые размеры
            buffer_primitives = list(filter(lambda primitive: primitive["Primitive_volume"] == similar_shapes, self.primitives_list))
            # Если не соберется хотя бы 2 переходим к следующему набору примитивов
            if len(buffer_primitives) >= 2:
                res = self._complete_correlation(buffer_primitives, year_of_correlation=table_data_slices, log_show=show_logs)
                self.correlation_results.append(res)
            else:
                if self.warning_logs:
                    print(f"Cannot find a pair (similar shape) to {[primitive['Primitive_name'] for primitive in buffer_primitives]}. Shape={similar_shapes}")
                continue

    def _complete_correlation(self, similar_primitives, year_of_correlation, log_show=True):
        if self.warning_logs:
            print('=' * 10)
        correlation_dict = {'Primitives_list': [primitive['Primitive_name'] for primitive in similar_primitives], 'Shape': similar_primitives[0]["Primitive_volume"]}
        if log_show:
            print('-' * 5 + '   EXTEND_INFO    ' + '-' * 5)
            print('Primitives total:', len(similar_primitives))
            print('Primitives objects:', [primitive['Primitive_name'] for primitive in similar_primitives])
            print('Primitives shape:', similar_primitives[0]["Primitive_volume"])
            print('-' * 5 + '   END_EXTEND_INFO    ' + '-' * 5)

        results = list()
        # Вырезаем из данных куски на которых будем считать корреляцию. Скорее всего будет корректно работать для разбитий любой ширины
        for borders in year_of_correlation:
            extend_info=False
            if log_show:
                extend_info = True
                print("Current Borders:", borders)
            #   Собирается список из "разбитых" примитивов
            work_primitives = [{"Primitive_name": primitive["Primitive_name"],"Primitive_content": primitive["Primitive_content"].loc[borders[0]: borders[1]], "Primitive_Freq": primitive["Primitive_Frequency"]} for primitive in similar_primitives]
            # Проверяется условие на наличие данных в примитиве
            if work_primitives[0]["Primitive_content"].shape[0] == 0:
                continue

            # Проверка что размер всех "разбитых" примитивов совпадает
            if len(np.unique([work_primitive["Primitive_content"].shape[0] for work_primitive in work_primitives])) > 1:
                continue

            carretes, corrs =  self._calculate_correlation(work_primitives, extend_info=extend_info, borders=borders)

            results.append({"borders": borders, "timesum": carretes, "corr_tables": corrs})
        correlation_dict['results'] = results
        return correlation_dict


    def _calculate_correlation(self, primitives, borders, list_return_summary=['5T', '30T', '1H', '6H', '1D'],extend_info=True):
        return_correlations = list()
        # Логарифмируем, считаем доходность, ресемплим в зависимости от входной частоты данных
        # Тут можно выбрать с каким столбцом будем работать. Сейчас выбран столбец open
        logged_primitives = [[primitive["Primitive_name"], np.log(primitive["Primitive_content"].open).diff()[1:].resample(primitive["Primitive_Freq"]).first()] for primitive in primitives]
        # Переменная зависящая от количества объектов в "пачке". Нужна для адекватной отрисовки heatmapa
        WIDTH_CONST = 2 * len(primitives)
        if self.graphs:
            fig, ax = plt.subplots(1, len(list_return_summary), figsize=(len(list_return_summary) * WIDTH_CONST ,WIDTH_CONST))
            fig.suptitle(f"{datetime.date.strftime(borders[0], '%Y-%m')}:{datetime.date.strftime(borders[1], '%Y-%m')}")

        # Меняем частоту данных для взятия корреляции за различные временные промежутки
        for number, research_carret in enumerate(list_return_summary):
            if extend_info:
                print('Current Research Resample:', research_carret)

            corr_df = pd.DataFrame()
            # Создаем массив из примитивов преобразованных под временные промежутки
            resampled_primitives = [[primitive[0], primitive[1].resample(research_carret).sum()] for primitive in logged_primitives]
            for obj in resampled_primitives:
                corr_df[f"{obj[0]}"] = obj[1]
            correlation = corr_df.corr()
            # Отрисовка
            if self.graphs:
                sns.heatmap(correlation, ax=ax[number], annot=True, fmt=".2g")
                ax[number].title.set_text(f"{research_carret}")

            return_correlations.append(correlation)
        return list_return_summary, return_correlations

    def unstable_matrix(self):
        # Очень некрасивый метод. Но пока что так
        for buff in self.correlation_results:
            big = pd.DataFrame()
            #print(buff["results"])
            # Проверяем что удалось разбить данные на хотя бы 2 "пачки". Иначе нельзя говорить о стабильности во времени
            if len(buff["results"]) == 1:
                if self.warning_logs:
                    print(f'No sense to calculate unstable factor for {buff["Primitives_list"]}. \n It has only one object inside. Skip...')
                continue
            if len(buff["results"]) > 1:
                # Очень плохой кусок кода, пока что не понял как его можно поправить чтобы работало
                # DONT TOUCH
                timesum = buff["results"][0]["timesum"]
                _obj = list()
                for i in range(len(timesum)):
                    _return = list()
                    for _b in buff["results"]:
                        _return.append(list(filter(lambda x: x[0] == timesum[i], list(zip(_b["timesum"], _b["corr_tables"])))))
                    _obj.append((timesum[i], _return))
                # DONT TOUCH

                results = list()

                # Связываю все матрицы корреляций за все доступные временные промежутки. Готовый метод не нашел, поэтому написал свое очень некрасивое решение
                # Нужно поправить чтобы пробегаться только по верхнетреугольной матрице.
                for _ in _obj[:]:
                    array_of_df = list()
                    for __ in _[1]:
                        array_of_df.append(__[0][1])

                    #   So dumb... sry
                    i,j = array_of_df[0].shape
                    pdf = pd.DataFrame()
                    for _ in range(i):
                        for __ in range(j):
                            elements = [df.iloc[_,__] for df in array_of_df]
                            # print(array_of_df[0].index[_])
                            # print(f"i={_}, j={__}, std={np.std(elements) / np.mean(elements)}")
                            #   Calculate CV Ratio
                            #   Стандартное отклонение на модуль среднего. Как я понимаю это хоть как-то нормализует показатель
                            pdf.loc[array_of_df[0].index[_], array_of_df[0].index[__]] = np.std(elements) / abs(np.mean(elements))

                    results.append(pdf)
                #   Сверху я считаю показатель для всех частот разбития (пример: 5минут, 10минут, 30минут, 1час) для которых мы считаем корреляцию.
                #   Этот показатель берется из выборки корреляций для разных временных промежутков (пример: 2018-2019;2019-2020;2020-2021).
                #   results.append(pdf) собирает таблицу устойчивости для данных по примеру:
                #   Returns за 10минут для 2019, 2020, 2021 дают корреляции за [P_10min_2019, P_10min_2020, P_10min_2021] по ним и считаем устойчивость на 10минутной частоте returns
                #   Снизу усредняются показатели устойчивости за все частоты разбиения возвратов
                big = pd.concat(results)
                big = pd.pivot_table(big, index=big.index, aggfunc={np.mean})
            self.total_result.append(big)


In [150]:
years = [2019, 2020, 2021]
#years = [2019]
#   Тут можно задать границы по которым будут резаться данные
borders_list = [[datetime.datetime(year=x, month=1, day=1), datetime.datetime(year=x+1, month=1, day=1)] for x in years]


data_path = glob('zip_data/*.csv')[:]
TestCorr = CorrelationTables(show_logs=False, show_graphs=False)
for _ in data_path:
    TestCorr.add_research_primitive(_)

TestCorr.make_correlation_table(table_data_slices=borders_list, show_logs=False)
TestCorr.unstable_matrix()
TestCorr.total_result

Please add data primitives in format of data path
No sense to calculate unstable factor for ['NQ-21', 'ES-21']. 
 It has only one object inside. Skip...


[          AUDCAD    AUDUSD    EURCHF    EURGBP    EURUSD    GBPJPY    USDCHF  \
             mean      mean      mean      mean      mean      mean      mean   
 AUDCAD  0.000000  0.093794  0.612686  0.645316  0.216511  0.432266  0.337272   
 AUDUSD  0.093794  0.000000  0.457008  0.411396  0.172067  0.124779  0.295504   
 EURCHF  0.612686  0.457008  0.000000  0.724673  0.405937  0.194127  0.324640   
 EURGBP  0.645316  0.411396  0.724673  0.000000  0.159863  0.054065  0.185829   
 EURUSD  0.216511  0.172067  0.405937  0.159863  0.000000  0.099763  0.084669   
 GBPJPY  0.432266  0.124779  0.194127  0.054065  0.099763  0.000000  2.317176   
 USDCHF  0.337272  0.295504  0.324640  0.185829  0.084669  2.317176  0.000000   
 USDJPY  1.350700  1.321030  0.409411  0.940081  0.205214  0.143183  0.052682   
 
           USDJPY  
             mean  
 AUDCAD  1.350700  
 AUDUSD  1.321030  
 EURCHF  0.409411  
 EURGBP  0.940081  
 EURUSD  0.205214  
 GBPJPY  0.143183  
 USDCHF  0.052682  
 USDJPY 

[{'Primitives_list': ['NQ-21', 'ES-21'],
  'Shape': 401,
  'results': [{'borders': [datetime.datetime(2021, 1, 1, 0, 0),
     datetime.datetime(2022, 1, 1, 0, 0)],
    'timesum': ['5T', '30T', '1H', '6H', '1D'],
    'corr_tables': [          NQ-21     ES-21
     NQ-21  1.000000  0.884906
     ES-21  0.884906  1.000000,
               NQ-21     ES-21
     NQ-21  1.000000  0.876161
     ES-21  0.876161  1.000000,
               NQ-21     ES-21
     NQ-21  1.000000  0.788137
     ES-21  0.788137  1.000000,
               NQ-21     ES-21
     NQ-21  1.000000  0.965953
     ES-21  0.965953  1.000000,
            NQ-21  ES-21
     NQ-21    1.0   -1.0
     ES-21   -1.0    1.0]}]},
 {'Primitives_list': ['AUDCAD',
   'EURUSD',
   'USDCHF',
   'EURGBP',
   'EURCHF',
   'USDJPY'],
  'Shape': 1107391,
  'results': [{'borders': [datetime.datetime(2019, 1, 1, 0, 0),
     datetime.datetime(2020, 1, 1, 0, 0)],
    'timesum': ['5T', '30T', '1H', '6H', '1D'],
    'corr_tables': [          AUDCAD    EURU